In [ ]:
# 读取谷歌云盘的数据请求
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=False)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
!pip install pyts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 10.0 MB/s 


In [3]:
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField

In [ ]:
# focus on file
# '12k Drive End Bearing Fault Data'
# '12k Fan End Bearing Fault Data'
# '48k Drive End Bearing Fault Data'

'''
    Generate data for testing.
    Clip 100 1000-length samples. And random clip 13 256-length samples. To file time series.
'''

import os, sys

import scipy.io as io
import seaborn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

# return the cantegory of file idx.mat, with the joint of faultplace and fault diameter
def getcan(idx):
    for i,place in enumerate(fault_place):
        if idx in place:
            if i==0:ans='Inner'
            elif i==1:ans='Ball'
            else:ans='Outer'
            break
    else:return None
    for i,fd in enumerate(FD):
        if idx in fd:
            if i==0:ans+='_7'
            elif i==1:ans+='_14'
            elif i==2:ans+='_21'
            else:ans+='_28'
            break
    return ans

rootpath='/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Data for testing/Time Series'

for setting in ['12k Drive End Bearing Fault Data', '12k Fan End Bearing Fault Data', '48k Drive End Bearing Fault Data']:

    print(setting)
    if setting == '12k Drive End Bearing Fault Data':
        # hp=0
        Inner = [105, 169, 209, 3001]
        Ball = [118, 185, 222, 3005]
        Outer = [130, 197, 234]
        FD_7 = [105, 118, 130]
        FD_14 = [185, 197, 169]
        FD_21 = [209, 222, 234]
        FD_28 = [3001, 3005]
    elif setting == '12k Fan End Bearing Fault Data':
        Inner = [278, 274, 270]
        Ball = [282, 286, 290]
        Outer = [294, 313, 315]
        FD_7 = [278, 282, 294]
        FD_14 = [274, 286, 313]
        FD_21 = [270, 290, 315]
        FD_28 = []
    elif setting == '48k Drive End Bearing Fault Data':
        Inner = [109, 174, 213]
        Ball = [122, 189, 226]
        Outer = [135, 201, 238]
        FD_7 = [109, 122, 135]
        FD_14 = [174, 189, 201]
        FD_21 = [213, 226, 238]
        FD_28 = []

    fault_place = list(map(set, (Inner, Ball, Outer)))
    FD = list(map(set, (FD_7, FD_14, FD_21, FD_28)))

    path = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/' + setting
    datafiles = os.listdir(path)
    # category and the data belong to it
    cantegory = dict()
    for i in range(len(datafiles)):
        data = datafiles[i]
        file = getcan(int(data[:-4]))
        if not file: continue
        if file not in cantegory:
            cantegory[file] = []
        data = io.loadmat(os.path.join(path, data))
        for key in data.keys():
            if 'DE_time' in key:
                break
        else:
            raise 'No \'DE_time\''
        # print(d[key])
        timeseries = data[key]
        length = len(timeseries)
        for j in range(100):
            # generate time series samples first
            cantegory[file].append(timeseries[100 * j:100 * j + 1000])


    path = rootpath + setting
    print(len(cantegory))
    for key in cantegory:
        print(key, ':', len(cantegory[key]))
        for i, arr in enumerate(cantegory[key]):
            # Generate 13 256-len samples for a 1000-len time series
            for k in range(13):
                start = random.randint(0, 1000-260)
                arr_k = arr[start:start+256]
                io.savemat(os.path.join(path, key + '_' + str(i) + '_' + str(k) + '.mat'), {'data': arr_k})

12k Drive End Bearing Fault Data
11
Inner_7 : 100
Inner_14 : 100
Inner_21 : 100
Inner_28 : 100
Ball_7 : 100
Ball_14 : 100
Ball_21 : 100
Ball_28 : 100
Outer_14 : 100
Outer_21 : 100
Outer_7 : 100
12k Fan End Bearing Fault Data
9
Inner_7 : 100
Inner_14 : 100
Inner_21 : 100
Ball_7 : 100
Ball_14 : 100
Ball_21 : 100
Outer_7 : 100
Outer_14 : 100
Outer_21 : 100
48k Drive End Bearing Fault Data
9
Inner_7 : 100
Inner_14 : 100
Inner_21 : 100
Ball_7 : 100
Ball_14 : 100
Ball_21 : 100
Outer_7 : 100
Outer_14 : 100
Outer_21 : 100
